# 05 Feature space

## TODO

- Sæt labels på x og y aksen
- QOL lav nearst neighbor lidt pænere

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import fasttext

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

from tabulate import tabulate

In [2]:
data = pd.read_csv('./data/sklearn.csv')

model = fasttext.load_model('./data/fasttext_trained_model.bin')
data['vec'] = data['review'].apply(lambda x: model.get_sentence_vector(x))

In [99]:
def nearest_neighbours(*args):
    numbers_of_args= len(args)
    
    listDict = {}
    for i in range(1, numbers_of_args+1 ):
        listDict["Title_"+str(i)] = []
    
        for a in args:
            print(a[:])
            listDict["Title_"+str(1)].append(model.get_nearest_neighbors(a[: :1]))
#     title={}
#     neighbours=[]
#     words=[]
    
#     for t in range(len(args)):
#         title["args{0}".format(t)] = []
      

    
#     for k in neighbours[0]:
#         words.append(k)
        
#     print(words)
#     print(len(neighbours))
    print(listDict)
      
nearest_neighbours("dårlig", "fedt")

dårlig
fedt
dårlig
fedt
{'Title_1': [[(0.9993193745613098, 'd.'), (0.9987528324127197, ',Elendig'), (0.9984056353569031, 'bliver'), (0.9982389211654663, 'må'), (0.997963547706604, 'ringer'), (0.997402012348175, 'selvom'), (0.997389554977417, 'firma'), (0.9972306489944458, 'ALDRIG'), (0.9971181750297546, 'mails'), (0.996974527835846, 'pludselig')], [(0.9766262173652649, 'leverings'), (0.976620614528656, 'fint.'), (0.9755644798278809, 'præcis'), (0.9727078676223755, ',Fin'), (0.9717280268669128, 'leveringstid'), (0.9717122316360474, ',Nem'), (0.9715535640716553, 'dog'), (0.971206545829773, 'returlabel'), (0.9710429906845093, 'rimelig'), (0.9687397480010986, 'pakker.')], [(0.9993193745613098, 'd.'), (0.9987528324127197, ',Elendig'), (0.9984056353569031, 'bliver'), (0.9982389211654663, 'må'), (0.997963547706604, 'ringer'), (0.997402012348175, 'selvom'), (0.997389554977417, 'firma'), (0.9972306489944458, 'ALDRIG'), (0.9971181750297546, 'mails'), (0.996974527835846, 'pludselig')], [(0.976626

In [85]:
def nearest_neighbors(arg1, arg2, arg3):
    title1 = arg1
    title2 = arg2
    title3 = arg3
    a=[] 
    b=[] 
    c=[]
    arg1 = model.get_nearest_neighbors(arg1)
    arg2 = model.get_nearest_neighbors(arg2)
    arg3 = model.get_nearest_neighbors(arg3)
    for k, v in arg1:
        a.append(v)
    
    for k, v in arg2:
        b.append(v)
    
    for k, v in arg3:
        c.append(v)
    nearest_neighbors_list = pd.DataFrame(
    {
    title1: a,
    title2: b,
    title3: c
    }) 
    return nearest_neighbors_list.head(10)  
    
nearest_neighbors('dårlig', 'nogenlunde', 'penge')

,dårlig,nogenlunde,penge
0,d.,Samtidigt,nu
1,",Elendig",kritik!,intet
2,bliver,inkasso,vi
3,må,steder.,kundeservice
4,ringer,kontakte,igen.
5,selvom,App,trukket
6,firma,tre,modtaget
7,ALDRIG,varer.,jer
8,mails,saga,ja
9,pludselig,reserveret,ingen


In [ ]:
# print('choose three words')
# en = input()
# to = input()
# tre= input()

# nearest_neighbors(en, to, tre)

In [ ]:
kmeans = KMeans(n_clusters=5)
data['cluster'] = kmeans.fit_predict(data['vec'].values.tolist())

In [ ]:
pca = PCA(n_components=3)
data['x'] = pca.fit_transform(data['vec'].values.tolist())[:,0]
data['y'] = pca.fit_transform(data['vec'].values.tolist())[:,1]
data['z'] = pca.fit_transform(data['vec'].values.tolist())[:,2]

scatter = plt.scatter(data['x'], data['y'], c=data['cluster'], cmap='rainbow', alpha=0.5)

# dc = data_cluster
# cc = cluster_colors
# cl = cluster_labels
print(data['cluster'].value_counts())
dc = data['cluster'].value_counts()
dc = -np.sort(-dc)
cc = data['cluster'].value_counts().index.tolist()

cl = []
for i in model.labels:
    cl.append(i.replace('__',''))

handles = scatter.legend_elements(num=cc)[0]  # extract the handles from the existing scatter plot

plt.legend(title='Ratings', handles=handles,  
           labels=[cl[0]+' '+str(dc[0]),
                   cl[1]+' '+str(dc[1]),
                   cl[2]+' '+str(dc[2]),
                   cl[3]+' '+str(dc[3]),
                   cl[4]+' '+str(dc[4])])

In [ ]:
fig = plt.figure(1, figsize=(10,10))
ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)
ax.scatter(data['x'],data['y'],data['z'], c=data['cluster'], cmap='rainbow', alpha=0.5)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")
ax.set_facecolor('white')
plt.title("Review Clustering using K Means", fontsize=14)
plt.legend(title='Classes', handles=handles, 
           labels=[cl[0]+' '+str(dc[0]),
                   cl[1]+' '+str(dc[1]),
                   cl[2]+' '+str(dc[2]),
                   cl[3]+' '+str(dc[3]),
                   cl[4]+' '+str(dc[4])])